In [196]:
import numpy as np
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup

### Take a glance on data

In [657]:
gen_studio = pd.read_csv("genstudio.csv")

/Users/tugidon/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [138]:
gen_studio.shape

(2000000, 20)

In [658]:
metadata = pd.read_csv("metadata.csv")

In [140]:
metadata.shape

(841, 4)

In [659]:
gen_studio.head()

,Unnamed: 0,SNP Name,SNP Index,SNP Aux,Sample ID,SNP,Allele1 - Top,Allele2 - Top,Allele1 - Forward,Allele2 - Forward,Allele1 - AB,Allele2 - AB,Chr,Position,GC Score,GT Score,Theta,R,B Allele Freq,Log R Ratio
0,0,1_10573221,1,0,202341831114R01C01,[T/C],-,-,-,-,-,-,1,10573221,0.0000,0.0000,0.942,0.413,1.0000,0.4040
1,1,1_10673082,2,0,202341831114R01C01,[T/C],A,A,T,T,A,A,1,10673082,0.8272,0.8076,0.039,0.968,0.0000,0.3017
2,2,1_10723065,3,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,10723065,0.8316,0.8107,0.011,1.577,0.0000,0.0388
3,3,1_11337555,4,0,202341831114R01C01,[A/G],A,A,T,T,A,A,1,11337555,0.3781,0.7925,0.045,1.104,0.0000,0.2761
4,4,1_11407894,5,0,202341831114R01C01,[A/G],G,G,G,G,B,B,1,11407894,0.9038,0.8670,0.983,1.122,0.9994,0.0022


In [660]:
gen_studio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 20 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   SNP Name           object 
 2   SNP Index          int64  
 3   SNP Aux            int64  
 4   Sample ID          object 
 5   SNP                object 
 6   Allele1 - Top      object 
 7   Allele2 - Top      object 
 8   Allele1 - Forward  object 
 9   Allele2 - Forward  object 
 10  Allele1 - AB       object 
 11  Allele2 - AB       object 
 12  Chr                object 
 13  Position           object 
 14  GC Score           float64
 15  GT Score           float64
 16  Theta              float64
 17  R                  float64
 18  B Allele Freq      float64
 19  Log R Ratio        float64
dtypes: float64(6), int64(3), object(11)
memory usage: 305.2+ MB


In [661]:
metadata.head()

,Unnamed: 0,dna_chip_id,breed,sex
0,0,202290551164R09C01,Д,Хр
1,1,202341831114R02C01,Д,Хр
2,2,202341831114R03C01,Д,Хр
3,3,202341831114R04C01,Д,Хр
4,4,202290551140R01C01,Д,Хр


In [144]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   841 non-null    int64 
 1   dna_chip_id  841 non-null    object
 2   breed        841 non-null    object
 3   sex          841 non-null    object
dtypes: int64(1), object(3)
memory usage: 26.4+ KB


### Creating database 

In [662]:
connection = sqlite3.connect("gene.db")

###  Adding genesstudio data

In [663]:
query = '''CREATE TABLE genes(
                                snp_id INTEGER PRIMARY KEY,
                                snp_name TEXT,
                                snp_index INTEGER,
                                snp_aux INTEGER,
                                sample_id TEXT,
                                snp TEXT,
                                allele_1_top TEXT,
                                allele_2_top TEXT,
                                allele_1_forward TEXT,
                                allele_2_forward TEXT,
                                allele_1_ab TEXT,
                                allele_2_ab TEXT,
                                chr TEXT,
                                position TEXT,
                                gc_score FLOAT,
                                gt_score FLOAT,
                                theta FLOAT,
                                r FLOAT,
                                b_allele_freq FLOAT,
                                log_r_ratio FLOAT
                                )'''

In [664]:
connection.execute(query)

In [665]:
connection.commit()

### Fill database with genstudio data

In [666]:
fill_data = gen_studio.iloc[:, 1:]

In [667]:
new_names = [i.replace(" ", "_") for i in fill_data.columns]

In [668]:
fill_data.columns = new_names

In [669]:
fill_data_list = fill_data.values.tolist()

In [670]:
insertion_query = """ INSERT INTO genes(
                             snp_name, snp_index, 
                             snp_aux, sample_id,
                             snp, allele_1_top,
                             allele_2_top, allele_1_forward,
                             allele_2_forward, allele_1_ab,
                             allele_2_ab,chr,
                             position, gc_score,
                             gt_score,theta,
                             r, b_allele_freq,
                             log_r_ratio)
                             VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""

In [671]:
for i in fill_data_list:
    connection.execute(insertion_query, i)

In [672]:
connection.commit()

### Adding metadata to database

In [673]:
query = '''CREATE TABLE metadata(
                                snp_id INTEGER PRIMARY KEY,
                                dna_chip_id TEXT,
                                breed TEXT,
                                sex TEXT
                                )'''

In [674]:
connection.execute(query)

In [675]:
connection.commit()

In [676]:
insertion_query = """ INSERT INTO metadata(
                             dna_chip_id, breed, sex)
                             VALUES(?,?, ?)"""

In [677]:
metadata = metadata.iloc[:, 1:]

In [678]:
metadata_list = metadata.values.tolist()

In [679]:
for i in metadata_list:
    connection.execute(insertion_query, i)

In [680]:
connection.commit()

### Selecting from database 

In [681]:
select_query = '''SELECT snp_name, snp_index
                    FROM genes'''

In [682]:
result = connection.execute(select_query).fetchmany(5)

In [683]:
result

[('1_10573221', 1),
 ('1_10673082', 2),
 ('1_10723065', 3),
 ('1_11337555', 4),
 ('1_11407894', 5)]

### Parsing

In [259]:
link = "https://www.film.ru/soon"

In [260]:
resp = requests.get(link)

In [262]:
soup = BeautifulSoup(resp.text)

In [286]:
lists = soup.find("div", id = "list_content")

Code was being written adaptively to output and may not work ideally if new requests change

In [589]:
countries= ["США", "Гонконг", "Таиланд", "Япония", "Франция", "Польша", "Великобритания", "Швейцария", \
           "Дания", "Швеция", "Нидерланды", "Россия", "Лихтенштейн", "Бельгия", "ПуэртоРико", "Германия", \
           "Италия", "Норвегия", "РеспубликаКорея", "Чехия", "Канада", "Мексика", "Венесуэла", "Испания", \
           "Финляндия", "Венгрия", "Турция", "Китай", "Ирландия", "Австралия", "Индия", "Израиль"]

In [649]:
films = []
for div in lists.find_all("div", class_="film_catalog"):
    
    link = div.find("a") # link contains all subinformation
    description = link.text.split("\n")
    description = list(filter(lambda a: a != '', description)) # filter ''
    
    for i in range(4,len(description)):
        description[i] = description[i].replace(" ", "") # delete part of backspaces
        
    if description[1] == "Есть рецензия": # delete this section 
        description.pop(1)
        
    if "+" not in description[2]: # mark films without age rate
        description.insert(2, "-")
        

    eng_and_year = description[1].split(",") # proceess english name and year
    
    if eng_and_year[0].isdigit(): 
        description[1] = "-"
        description.insert(2, eng_and_year[0])
    else:
        description[1] = eng_and_year[0]
        description.insert(2, eng_and_year[1])
       
    janre_and_time = description[4].split(",") # process janre and time
    
    if not janre_and_time[-1].isalpha():
        description[4] = ",".join(janre_and_time[:-1])
        description.insert(5, janre_and_time[-1])
    elif len(janre_and_time) > 0:
        description[4] = ",".join(janre_and_time[:])
        description.insert(5, "-")
    else:
        description[4] = "-"
        description.insert(5, "-")   
        
    description[6] = description[6].replace(" ", "")
    country_and_produssor = description[6].split(",") # process country and produssor

    
    countries_select = []
    producers = []
    if country_and_produssor[0] == "film.ru:":
        description.insert(5,"-")
    for item in country_and_produssor:
        if item in countries:
            countries_select.append(item)
        else:
            producers.append(item)
    if len(countries) > 0:
        description[6] = ",".join(countries_select)
    else:
        description[6] = "-"
    if len(producers) > 0:
        description.insert(7,",".join(producers))
    else:
        description.insert(7,"-")
    
    description.pop(8) # delete names of rating systems
    description.pop(9)
    description.pop(10)
   
     
    
    films.append(description)    
   
    
    

In [650]:
connection = sqlite3.connect("films.db")

In [651]:
query = '''CREATE TABLE films(
                                snp_id INTEGER PRIMARY KEY,
                                rus_name TEXT,
                                eng_name TEXT,
                                year TEXT,
                                age_rate TEXT,
                                janres TEXT,
                                duration_min TEXT,
                                countries TEXT,
                                produssor TEXT,
                                film_ru_score TEXT,
                                IMDb_score TEXT,
                                audience_score TEXT
                                ) '''

In [652]:
connection.execute(query)

In [653]:
connection.commit()

In [654]:
insertion_query = ''' INSERT INTO films(
                             rus_name, eng_name,
                             year, age_rate, janres,
                             duration_min, countries,
                             produssor,film_ru_score,
                             IMDb_score, audience_score)
                             VALUES(?,?,?,?,?,?,?,?,?,?,?)'''

In [655]:
for i in films:
    connection.execute(insertion_query, i)

In [656]:
connection.commit()